In [ ]:
from keras.applications import inception_v3
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.layers import *
from keras.models import Model
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.utils import to_categorical
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
from keras.models import load_model
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import pickle

### 配置GPU

In [ ]:
# 配置GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # 使用编号为0号的GPU
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8 # 每个GPU现存上届控制在90%以内
session = tf.Session(config=config)
# 设置session
KTF.set_session(session)

#### 读取样本函数

In [ ]:
def read_img(img_id, data_dir, train_or_test, size):
    img = image.load_img(os.path.join(data_dir, train_or_test, '%s.jpg' % img_id), target_size=size)
    img = image.img_to_array(img)
    return img

### 单折模型小测试

In [ ]:
model = load_model("./pretrain_model/inceptionv3_model_5")

In [ ]:
test_img = read_img(31, './', 'train', (224,224))
xte = inception_v3.preprocess_input(np.expand_dims(test_img.copy(), axis=0))
res_list = model.predict(xte)
res_list

In [ ]:
res = np.argmax(res_list)
res

### 训练集测试

In [ ]:
label_train = pd.read_csv('./data/train_5.csv')
id_ = label_train['id'].values
label = label_train['label'].values

In [ ]:
flag = 0
for i,num in tqdm(enumerate(id_)):
    test_img = read_img(num, './', 'train', (224,224))
    xte = inception_v3.preprocess_input(np.expand_dims(test_img.copy(), axis=0))
    res = np.argmax(model.predict(xte))
    if res == label[i]:
        flag+=1
flag/len(id_)

### 测试集预测

In [ ]:
test_label = []
for i in tqdm(range(6671)):
    test_img = read_img(i, './', 'test', (224,224))
    xte = inception_v3.preprocess_input(np.expand_dims(test_img.copy(), axis=0))
    label = np.argmax(model.predict(xte))
    test_label.append(label)

### 模型5折自融合

#### 读取测试集

In [ ]:
testimg_book = []
for i in tqdm(range(6671)):
    test_img = read_img(i, './', 'test', (224,224))
    xte = inception_v3.preprocess_input(np.expand_dims(test_img.copy(), axis=0))
    testimg_book.append(xte)

In [ ]:
testimg_book[0].shape

In [ ]:
test_label_book = []
for i in range(1,6):
    model = load_model("./pretrain_model/inceptionv3_model_{}".format(i))
    test_label = []
    for img in tqdm(testimg_book):
        label = model.predict(img)
        test_label.append(label)
    test_label_book.append(test_label)

In [ ]:
merge_test_label = np.asarray(test_label_book[0])+np.asarray(test_label_book[1])+np.asarray(test_label_book[2])+np.asarray(test_label_book[3])+np.asarray(test_label_book[4])

In [ ]:
len(merge_test_label)

In [ ]:
merge_test_label = map(lambda x:np.argmax(x), merge_test_label)

In [ ]:
merge_test_label = list(merge_test_label)

In [ ]:
merge_test_label

In [ ]:
id_ = [x for x in range(6671)]
test_df = pd.DataFrame({'id':id_,'label':merge_test_label})
test_df.shape

In [ ]:
test_df.label[test_df['label']!=0].count()

In [ ]:
test_df.to_csv('test.csv', header=0, index=0)